In [ ]:
from scipy.stats import norm
from scipy.stats import t as tdist
from scipy.optimize import minimize
import numpy as np

In [ ]:
#使用正态分布对数据进行拟合
def fit_normal(x):
    """
    使用正态分布拟合数据。

    参数:
        x (numpy.ndarray): 输入数据。

    返回:
        dict: 包含正态分布参数和误差模型的字典。
    """
    m = np.mean(x)
    s = np.std(x)
    error_model = norm(loc=m, scale=s)

    # 计算误差和分位数
    errors = x - m
    u = error_model.cdf(x)

    return {
        "mu": m,
        "sigma": s,
        "error_model": error_model,
        "errors": errors,
        "u": u,
        "eval": lambda u, mu=m, s=s: norm(loc=mu, scale=s).ppf(u)  # 根据分位值生成样本
    }

In [ ]:
def fit_general_t(x):
    start_m = np.mean(x)
    start_nu = 6.0 / ((np.mean(x ** 4) / (np.var(x) ** 2)) - 3) + 4
    start_s = np.sqrt(np.var(x) * (start_nu - 2) / start_nu)
    def general_t_ll(params):
        mu, s, nu = params
        if s < 1e-6 or nu < 2.0001:
            return np.inf
        pdf_values = tdist.pdf((x - mu) / s, df=nu) / s
        return -np.sum(np.log(pdf_values))
    initial_guess = [start_m, start_s, start_nu]
    bounds = [(None, None), (1e-6, None), (2.0001, None)]
    result = minimize(general_t_ll, initial_guess, bounds=bounds, method="L-BFGS-B")
    # Change: Assign the optimized values to mu, sigma, and nu directly
    mu, sigma, nu = result.x
    error_model = tdist(df=nu, loc=mu, scale=sigma)
    cdf_func = lambda y: error_model.cdf(y)
    quantile_func = lambda u: error_model.ppf(u)
    errors = x - mu
    u = cdf_func(x)
    def eval(u):
        return quantile_func(u)
    return {
        "mu": mu, # Change: Return 'mu' instead of 'm'
        "sigma": sigma, # Change: Return 'sigma' instead of 's'
        "nu": nu,
        "errors": errors,
        "u": u,
        "error_model": error_model,
        "cdf_func": cdf_func,
        "quantile_func": quantile_func,
        "eval": eval
    }

In [ ]:
def fit_regression_t(y, x):
    X = np.hstack([np.ones((len(x), 1)), x])
    b_start = np.linalg.inv(X.T @ X) @ (X.T @ y)
    residuals = y - X @ b_start
    start_m = np.mean(residuals)
    start_nu = 6.0 / ((np.mean(residuals ** 4) / np.var(residuals) ** 2) - 3) + 4
    start_s = np.sqrt(np.var(residuals) * (start_nu - 2) / start_nu)
    def negative_log_likelihood(params):
        m, s, nu = params[:3]
        beta = params[3:]
        residuals = (y - X @ beta - m) / s
        if s <= 0 or nu <= 2.0001:
            return np.inf
        log_likelihood = np.sum(tdist.logpdf(residuals, df=nu)) - len(y) * np.log(s)
        return -log_likelihood

    initial_params = np.concatenate(([start_m, start_s, start_nu], b_start))
    bounds = [(None, None), (1e-6, None), (2.0001, None)] + [(None, None)] * X.shape[1]
    result = minimize(negative_log_likelihood, initial_params, bounds=bounds, method="L-BFGS-B")

    mu, sigma, nu = result.x[:3]
    beta = result.x[3:]
    error_model = tdist(df=nu, loc=mu, scale=sigma)
    def eval_model(new_x, u):
        new_X = np.hstack([np.ones((new_x.shape[0], 1)), new_x])
        quantile_correction = error_model.ppf(u)
        return new_X @ beta + quantile_correction
    errors = y - eval_model(x, np.full(len(y), 0.5))
    u = error_model.cdf(errors)

    return {
        "mu": mu,
        "sigma": sigma,
        "nu": nu,
        "beta": beta,
        "error_model": error_model,
        "eval_model": eval_model,
        "errors": errors,
        "u": u
    }

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# 拟合 AR(1) 模型
model = ARIMA(returns, order=(1, 0, 0))
ar1_fit = model.fit()

# 打印拟合结果
print(ar1_fit.summary())

In [1]:
def simulate_ar1(ar1_fit, nSim, fwdT):
    coef = ar1_fit.params
    mu = coef["const"]  # 均值
    phi = coef["ar.L1"]  # AR(1) 系数
    sigma = np.sqrt(ar1_fit.resid.var())

    # 随机生成白噪声
    innovations = np.random.normal(0, sigma, size=(nSim, fwdT))
    sim_returns = np.zeros((nSim, fwdT))  # 初始化模拟结果矩阵

    # 按路径模拟未来收益
    for i in range(nSim):
        for t in range(fwdT):
            sim_returns[i, t] = (
                mu + phi * (sim_returns[i, t - 1] if t > 0 else 0) + innovations[i, t]
            )

    return sim_returns
#ar1_fit可以替换为其他的模型